<a href="https://colab.research.google.com/github/rahiakela/cs231n-convolutional-neural-networks-for-visual-recognition/blob/master/module-1-neural-networks/8_putting_it_together_minimal_neural_network_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Putting it together: Minimal Neural Network Case Study

Table of Contents:

- Generating some data
- Training a Softmax Linear Classifier
 - Initialize the parameters
 - Compute the class scores
 - Compute the loss
 - Computing the analytic gradient with backpropagation
 - Performing a parameter update
 - Putting it all together: Training a Softmax Classifier
- Training a Neural Network
- Summary

In this section we’ll walk through a complete implementation of a toy Neural Network in 2 dimensions. We’ll first implement a simple linear classifier and then extend the code to a 2-layer Neural Network. As we’ll see, this extension is surprisingly simple and very few changes are necessary.

## Generating some data